<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GS-for-NB" data-toc-modified-id="GS-for-NB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GS for NB</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-SVM" data-toc-modified-id="GS-for-SVM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GS for SVM</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-LR" data-toc-modified-id="GS-for-LR-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>GS for LR</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-RF" data-toc-modified-id="GS-for-RF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>GS for RF</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#VISUAL" data-toc-modified-id="VISUAL-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>VISUAL</a></span></li></ul></div>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Text-Classification/code")
!pip install pyLDAvis
!pip install gensim
!pip install pandas==1.3.0
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
nb_name = "Classification_svm_nb_baseline-upsampling+stem_voc_stage3_only_RF"

In [4]:
import numpy as np 
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.pipeline import Pipeline

from classification_utils import *
from clustering_utils import *
from eda_utils import *
from nn_utils_keras import *
from feature_engineering_utils import *

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [5]:
train, test = load_data(only_stem_voc=True)
train, upsampling_info = upsampling_train(train)

# train_text, train_label = train_augmentation(train, select_comb=[['text'], ['reply', 'reference_one']])
train_text, train_label = train_augmentation(train, select_comb=None)
test_text, test_label = test['text'], test['label']


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


In [6]:
####################################
### label mapper
####################################
labels = sorted(train_label.unique())
label_mapper = dict(zip(labels, range(len(labels))))
train_label = train_label.map(label_mapper)
test_label = test_label.map(label_mapper)
y_train = train_label.values
y_test = test_label.values

print(train_text.shape)
print(test_text.shape)
print(train_label.shape)
print(test_label.shape)

(11813,)
(7761,)
(11813,)
(7761,)


# GS for NB

In [12]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#               'clf__alpha': [0.6, 0.8, 1]}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"NB_cv_result_{nb_name}.xlsx")
set(y_test)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

## Best params

In [14]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# tfidf_vectorizer(train_text, test_text, min_df=2, max_df=100)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative 
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = MultinomialNB(alpha=0.6)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

roc_auc:  0.9695511685434128


,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,1.679302,0.098269,0.805265,0.934767,0.783345,0.930353,0.785632,0.930880,0.783345,0.930353
1,0.477964,0.088122,0.834692,0.930164,0.817812,0.924944,0.819379,0.925632,0.817812,0.924944
2,0.476066,0.091776,0.831642,0.931550,0.814765,0.926637,0.816281,0.927516,0.814765,0.926637
3,0.467035,0.088732,0.808896,0.931795,0.798171,0.927427,0.798574,0.927788,0.798171,0.927427


# GS for SVM

In [15]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#              'clf__penalty':['l2'], 'clf__C':[1,2,3]}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"SVC_cv_result_{nb_name}.xlsx")

## Best params

In [16]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=1.0, dual=True, max_iter=3000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
cv

num of words: 10593
Dimension reduction with truncate SVD:
   input columns with  10593
   output columns with  1000


/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kwargs['lwork'] = ret[-2][0].real.astype(numpy.int)
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, ch

X_train.shape (11813, 1000)
X_test.shape (7761, 1000)
classification_report:
f1:  {'micro avg': 0.7135678391959799, 'macro avg': 0.694355725553717} 

                          precision  recall  f1-score  support
alt.atheism                   0.629   0.564     0.595    319.0
comp.graphics                 0.652   0.607     0.628    389.0
comp.os.ms-windows.misc       0.616   0.612     0.614    394.0
comp.sys.ibm.pc.hardware      0.617   0.599     0.608    392.0
comp.sys.mac.hardware         0.656   0.668     0.662    385.0
comp.windows.x                0.694   0.630     0.660    395.0
misc.forsale                  0.776   0.833     0.803    390.0
rec.autos                     0.754   0.777     0.766    395.0
rec.motorcycles               0.892   0.869     0.880    398.0
rec.sport.baseball            0.690   0.874     0.771    397.0
rec.sport.hockey              0.975   0.809     0.884    827.0
sci.crypt                     0.868   0.828     0.848    396.0
sci.electronics               0

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,7.820221,0.019469,0.791926,0.970909,0.792485,0.970764,0.791100,0.970786,0.792485,0.970764
1,7.927754,0.017849,0.829050,0.967740,0.828649,0.967607,0.828509,0.967684,0.828649,0.967607
2,8.202814,0.018840,0.838718,0.968035,0.838131,0.967833,0.837889,0.967931,0.838131,0.967833
3,8.041472,0.019956,0.812254,0.968481,0.813749,0.968172,0.812785,0.968273,0.813749,0.968172


In [17]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=1.0, dual=True, max_iter=3000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
cv

num of words: 10593
X_train.shape (11813, 10593)
X_test.shape (7761, 10593)
classification_report:
f1:  {'micro avg': 0.7370184254606366, 'macro avg': 0.7207235578147012} 

                          precision  recall  f1-score  support
alt.atheism                   0.697   0.599     0.644    319.0
comp.graphics                 0.640   0.622     0.631    389.0
comp.os.ms-windows.misc       0.603   0.614     0.609    394.0
comp.sys.ibm.pc.hardware      0.606   0.620     0.613    392.0
comp.sys.mac.hardware         0.668   0.678     0.673    385.0
comp.windows.x                0.719   0.648     0.682    395.0
misc.forsale                  0.780   0.828     0.803    390.0
rec.autos                     0.779   0.795     0.787    395.0
rec.motorcycles               0.917   0.887     0.902    398.0
rec.sport.baseball            0.724   0.892     0.799    397.0
rec.sport.hockey              0.978   0.822     0.894    827.0
sci.crypt                     0.876   0.856     0.866    396.0
sci.elec

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,1.564910,0.104158,0.805742,0.996731,0.804333,0.996726,0.803657,0.996713,0.804333,0.996726
1,1.550299,0.098939,0.861524,0.997082,0.861158,0.997065,0.860905,0.997059,0.861158,0.997065
2,1.647649,0.107969,0.858818,0.997406,0.857433,0.997404,0.857560,0.997387,0.857433,0.997404
3,1.652449,0.100616,0.825632,0.997523,0.826956,0.997517,0.826220,0.997507,0.826956,0.997517


# GS for LR

In [18]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr', max_iter=1000))])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#              'clf__penalty':['l2'], 'clf__C':[1,2,3], 'clf__solver': ['liblinear', 'newton-cg', 'sag', 'saga' , 'lbfgs']}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"LR_1_cv_result_{nb_name}.xlsx")

In [19]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr'))])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#              'clf__penalty':['elasticnet'], 'clf__C':[1,2,3], 'clf__l1_ratio':[0.2, 0.5,0.8],'clf__solver': ['saga']}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"LR_2_cv_result_{nb_name}.xlsx")

## Best params

In [20]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LogisticRegression(penalty="l2", multi_class='ovr', max_iter=1000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

num of words: 10593
Dimension reduction with truncate SVD:
   input columns with  10593
   output columns with  1000


/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kwargs['lwork'] = ret[-2][0].real.astype(numpy.int)
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, ch

X_train.shape (11813, 1000)
X_test.shape (7761, 1000)


/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring(

classification_report:
f1:  {'micro avg': 0.7247777348279861, 'macro avg': 0.7053282712728359} 

                          precision  recall  f1-score  support
alt.atheism                   0.625   0.549     0.584    319.0
comp.graphics                 0.640   0.635     0.637    389.0
comp.os.ms-windows.misc       0.692   0.673     0.682    394.0
comp.sys.ibm.pc.hardware      0.652   0.617     0.634    392.0
comp.sys.mac.hardware         0.694   0.649     0.671    385.0
comp.windows.x                0.726   0.658     0.691    395.0
misc.forsale                  0.779   0.859     0.817    390.0
rec.autos                     0.817   0.780     0.798    395.0
rec.motorcycles               0.921   0.849     0.884    398.0
rec.sport.baseball            0.686   0.902     0.779    397.0
rec.sport.hockey              0.975   0.805     0.882    827.0
sci.crypt                     0.892   0.816     0.852    396.0
sci.electronics               0.610   0.606     0.608    393.0
sci.med              

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated.

cv average f1 macro:  0.8035443991003155
cv average f1 micro:  0.8036918264271602
roc_auc:  0.9667252358319611


,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,7.208867,0.018652,0.798226,0.899279,0.792146,0.896941,0.792417,0.896965,0.792146,0.896941
1,7.729173,0.019893,0.820563,0.893407,0.816458,0.890971,0.816467,0.890636,0.816458,0.890971
2,8.346344,0.019165,0.822697,0.894810,0.817812,0.892438,0.817953,0.892664,0.817812,0.892438
3,7.787139,0.019841,0.791379,0.894421,0.788351,0.892438,0.787340,0.892359,0.788351,0.892438


# GS for RF

In [22]:
from xgboost import XGBClassifier

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), 
                     ('clf', XGBClassifier(objective='multi:softmax', n_estimators=500, num_class=20, learning_rate=0.075, \
                                           colsample_bytree=0.7, subsample=0.8, eval_metric='merror'))])

parameters = {
    'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
    #'svd__n_components': (250, 500, 750),
    #'clf__n_estimators': (250, 500, 750),
    'clf__max_depth': (4, 6, 8),
    'clf__min_child_weight': (1, 5, 10),
    #'clf__alpha': (0.00001, 0.000001),
    #'clf__penalty': ('l2', 'elasticnet'),
    #'clf__max_iter': (10, 50, 80),
}

gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"RF_1_cv_result_{nb_name}.xlsx")

## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative 
X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


param = {'max_depth': 2, 'objective': 'multi:softmax', 'n_estimators': 2, 
         'booster':'gblinear', # linear model
         'alpha': 0.0001,  # alpha is the L1 regularizer
         'lambda': 1} # lambda is the L2 regularizer
param['num_class'] = len(labels)
param['eta'] = 0.1 # # scale weight of positive examples
param['eval_metric'] = ['auc', 'ams@0']
clf = xgb.XGBClassifier(param)

In [ ]:
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
classification_report = evaluation_report(y_test, pred, labels=labels)
# cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
# pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

In [ ]:
# clf.save_model("xgb.bin")
# clf.load_model("xgb.bin")
xgb.plot_importance(clf)
xgb.to_graphviz(clf, num_trees=2)

# VISUAL

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import precision_recall_fscore_support, classification_report, roc_curve, auc, precision_recall_curve
# mapping from integer feature name to original token string
feature_names = vectorizer.get_feature_names()

# Feature reduction with Kbest features based on chi2 score
ch2 = SelectKBest(chi2, k=50000)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)
if feature_names:
    # keep selected feature names
    feature_names = [feature_names[i] for i in ch2.get_support(indices=True)]

In [ ]:

p = (clf.predict_proba(test_dtm))[:, 1] 
p_binary = clf.predict(test_dtm)

p_label = 2
fpr, tpr, thresholds = roc_curve(test_y, p, pos_label=p_label)
auc_score = auc(fpr, tpr)

pre, rec, thresholds = precision_recall_curve(test_y, p, pos_label=p_label)

# calculate auc
prc_score = auc(rec, pre)

if show_plots:

    print(classification_report(test_y, p_binary))

    plt.subplot(1, 2, 1)

    plt.plot(fpr, tpr, color='darkorange', lw=2)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('AUC')

    plt.subplot(1, 2, 2)
    plt.plot(rec, pre, color='darkorange', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PRC')
    plt.tight_layout()

    plt.show()

print("AUC: {:.2%}, PRC: {:.2%}".format(auc_score, prc_score))
